In [1]:
import os

In [2]:
%pwd #checking in which folder i am in

'c:\\Users\\nikit\\Downloads\\projects\\mlops-end-to-end\\lax-model\\research'

In [3]:
os.chdir("../") #Going back only one folder or to main folder
%pwd

'c:\\Users\\nikit\\Downloads\\projects\\mlops-end-to-end\\lax-model'

In [4]:
#This is the entity that will be going in the config_entity
from dataclasses import dataclass
from pathlib import Path

#Working with this to configure the entity
@dataclass(frozen=True)
class DataIngestionConfig: #return type of the function defined in the config.yml file
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from laxProject.constants import *
from laxProject.utils.common import read_yaml, create_directories

In [6]:
#For configuration manager in the config
class ConfigurationManager: #returning from the common.py file
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig: #Reading from config.yaml file and the data class for return
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
#This would be for the components
import os
import urllib.request as request
import zipfile
from laxProject import logger
from laxProject.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
#For the pipeline code
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-19 21:43:42,154: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-02-19 21:43:42,257: INFO: common: YAML file: params.yaml loaded successfully]
[2025-02-19 21:43:42,260: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-02-19 21:43:42,262: INFO: common: created directory at: artifacts]
[2025-02-19 21:43:42,263: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-19 21:43:42,682: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D714:3F7205:F373FC:10E615B:67B696DC
Accept-Ranges: bytes
Date: Thu, 2